In [ ]:
import cv2 as cv
import mediapipe as mp
import numpy as np
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import matplotlib.pyplot as plt
from scipy.spatial import distance

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def get_middle(img, landmark):
    
       
    # results.pose_landmarks 에 감지한 관절의 좌표가 저장되어 있다.
        
    # 왼쪽, 오른쪽 어깨 관절의 중심점 찾기
    x_LEFT_SHOULDER = landmark[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x*img.shape[1]
    y_LEFT_SHOULDER = landmark[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y*img.shape[0]
    z_LEFT_SHOULDER = landmark[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z*100
        
    x_RIGHT_SHOULDER = landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x*img.shape[1]
    y_RIGHT_SHOULDER = landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y*img.shape[0]
    z_RIGHT_SHOULDER = landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].z*100
    
    x_MIDDLE_SHOULDER = (x_LEFT_SHOULDER + x_RIGHT_SHOULDER) / 2
    y_MIDDLE_SHOULDER = (y_LEFT_SHOULDER + y_RIGHT_SHOULDER) / 2
    z_MIDDLE_SHOULDER = (z_LEFT_SHOULDER + z_RIGHT_SHOULDER) / 2
    
    MIDDLE_SHOULDER = (int(x_MIDDLE_SHOULDER), int(y_MIDDLE_SHOULDER))
    
    return MIDDLE_SHOULDER

def calculate_angle(img, landmark):
    
    x_LEFT_SHOULDER = landmark[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x*img.shape[1]
    y_LEFT_SHOULDER = landmark[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y*img.shape[0]
    LEFT_SHOULDER = (x_LEFT_SHOULDER, y_LEFT_SHOULDER)
    
    x_RIGHT_SHOULDER = landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x*img.shape[1]
    y_RIGHT_SHOULDER = landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y*img.shape[0]
    RIGHT_SHOULDER = (x_RIGHT_SHOULDER, y_RIGHT_SHOULDER)
    
    
    NOSE = (landmark[mp_pose.PoseLandmark.NOSE].x*img.shape[1],
            landmark[mp_pose.PoseLandmark.NOSE].y*img.shape[0])
    
    ## 코 - 왼쪽 어깨 - 오른쪽 어깨 각도 계산
    vector_1 = [NOSE[i] - LEFT_SHOULDER[i] for i in range(len(NOSE))]    
    vector_2 = [RIGHT_SHOULDER[i] - LEFT_SHOULDER[i] for i in range(len(RIGHT_SHOULDER))] 
    norm1 = np.linalg.norm(vector_1)
    norm2 = np.linalg.norm(vector_2)
        
    # 두 벡터의 내적 계산
    dot_product_1 = np.dot(vector_1, vector_2)
        
    # 두 벡터 사이의 각도 계산 (라디안)
    cos_theta_1 = dot_product_1 / (norm1 * norm2)
    theta_rad_1 = np.arccos(cos_theta_1)
    
    # 라디안을 각도로 변환하여 반환 (도)
    theta_deg_LEFT = np.degrees(theta_rad_1)
    
    
    
    ## 코 - 오른쪽 어깨 - 왼쪽 어깨 각도 계산
    vector_3 = [NOSE[i] - RIGHT_SHOULDER[i] for i in range(len(NOSE))]    
    vector_4 = [LEFT_SHOULDER[i] - RIGHT_SHOULDER[i] for i in range(len(LEFT_SHOULDER))] 
    norm3 = np.linalg.norm(vector_3)
    norm4 = np.linalg.norm(vector_4)
        
    dot_product_2 = np.dot(vector_3, vector_4)
        
    cos_theta_2 = dot_product_2 / (norm3 * norm4)
    theta_rad_2 = np.arccos(cos_theta_2)
    
    theta_deg_RIGHT = np.degrees(theta_rad_2)
        
    return (theta_deg_LEFT, theta_deg_RIGHT)

def calculate_distance(img, landmark):
    
    MIDDLE_SHOULDER = get_middle(img, landmark)
    
    NOSE = (landmark[mp_pose.PoseLandmark.NOSE].x*img.shape[1],
            landmark[mp_pose.PoseLandmark.NOSE].y*img.shape[0])
    
    dist = distance.euclidean(NOSE, MIDDLE_SHOULDER)
    
    return int(dist)
    

In [ ]:

# video feed
cap = cv.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    # with ~ as : 파일 or 함수를 열고 with 내부의 코드가 실행이 끝나면 닫힘 
    # 즉 with 내부에서는 pose를 이용해서 mediapipe 사용가능
    
    while cap.isOpened():
        ret, frame = cap.read() #ret: 비디오가 정상적으로 불러와졌는지
                                #frame : 실제 비디오 정보
                                
        # Detection
        image = cv.cvtColor(frame, cv.COLOR_BGR2RGB) # cv는 비디오를 BGR로 받아오고 
                                                     # mediapipe는 RGB로 비디오를 다루기 때문에 변환해야함
        image.flags.writeable = False
        
        results = pose.process(image) # 실제로 관절 감지를 진행하는 부분
        
        
        image.flags.writeable = True
        image = cv.cvtColor(image, cv.COLOR_RGB2BGR)
        
        
        # 랜드마크 추출
        try:
            landmarks = results.pose_landmarks.landmark
        except:
            pass
        
        # 관절 그리기
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,66), thickness=2, circle_radius=2)
                                    )
        # results.pose_landmarks 에 감지한 관절의 좌표가 저장되어 있다.
    
        
        # 어깨 사이 중심점 그리기
        cv.circle(image, get_middle(image, landmarks), 6, (0, 0, 255), -1) # 중심은 빨간색 원으로 그림 
        
        print(calculate_angle(image, landmarks))
                             
        cv.imshow("Mediapipe feed", image)

        
        
        if cv.waitKey(1) == ord('q'): # q누르면 break
            break
        
               
    cap.release()
    cv.destroyAllWindows()
    
    